## **Web Scraping de TUCARRO.COM paginas particulares**

### **1. Importamos las respectivas librerias**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json

### **2. Hacemos el respectivo requerimiento a la pagina**

➡️ Tenemos que hacer la peticion de conexion a la pagina que vamos a scrapear

In [2]:
html_text = requests.get('https://articulo.tucarro.com.co/MCO-2526957066-mercedes-benz-cla-180-13-amg-line-2022-_JM#position%3D1%26search_layout%3Dgrid%26type%3Ditem%26tracking_id%3Ddd2134b9-b228-49f8-a6c2-33ab624ba167')

➡️ Necesitamos un indicador que nos diga si la conexion fue realmente efectiva, en este caso, **200** es el código que muestra una conexion exitosa.

🗒️ **Observación:** Para una mayor informacion, les dejo el siguiente link 🔗 https://developer.mozilla.org/es/docs/Web/HTTP/Status/200

In [3]:
status_code = html_text.status_code
print(status_code)

200


➡️ Ahora, vamos a convertir la url de la pagina en la que estamos situados en un valor de referencia que utilizaremos posteriormente

In [4]:
conexion_url = html_text.url
print(conexion_url)

https://articulo.tucarro.com.co/MCO-2526957066-mercedes-benz-cla-180-13-amg-line-2022-_JM#position%3D1%26search_layout%3Dgrid%26type%3Ditem%26tracking_id%3Ddd2134b9-b228-49f8-a6c2-33ab624ba167


### **3. Seleccionamos las grillas especificas de donde vamos a sacar la informacion**

In [5]:
soup = BeautifulSoup(html_text.text, 'lxml')

➡️ En esta pagina hay bastante informacion, sin embargo, la primera parte que dice *"Caracteristicas del producto"* pareciera estar ya contenida en las tablas que se muestran justo abajo. Por decision, vamos a tomar solo la informacion que se detalla en las tablas.

⚠️ **Alerta:** Esta parte de la pagina pareciera esta protegida para hacer *web scraping*, por lo tanto debemos excabar para ver que informacion si es posible extraer.

#### 3.1 Condiciones de compra

➡️ Exploremos la informacion que se esta extrayendo en el request

In [6]:
print(soup)

<!DOCTYPE html>
<html lang="es-CO">
<head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://www.google.com" rel="preconnect"/><link href="https://data.mercadolibre.com" rel="preconnect"/><link href="https://http2.mlstatic.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://analytics.tucarro.com.co" rel="preconnect"/><link href="https://analytics.tucarro.com" rel="preconnect"/><link href="https://analytics.mercadolibre.com" rel="preconnect"/><link href="https://www.google.com.co" rel="preconnect"/><script nonce="viQhB0Z7uMbAg5Q4uYl6ag==" type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRBR-766f4fb616d3a2368ce","applicationID":"1588198355","agentToken":null,"applicationTime":271.386801,"transactionName":"bgQDMEcFXkJZBkYNWldOJBxFFlVCSw9BS3J8NU5LDxRCXlwQURB8XUlJP1gYfWxjBB8edBQ7PB8HGWtERDBvWxwUSTpUGF1tGhFNGE

➡️ La unica informacion que pareciera estar disponible tien que ver con *Color*, *Tipo de carroceria*, *Marca*, etc. Vamos a extraer esa informacion. El formato esta en formato JSON asi que vamos a extraer la informacion teniendo este hecho presente.

In [7]:
# Buscamos en elemento script dentro del bloque del html
script = soup.find('script', type='application/ld+json')

# Extraer el contenido del script como texto. Strip se encargara de eliminar los espacios vacios al principio y al final de la cadena
script_content = script.text.strip()

# Convertir el contenido del script a un objeto JSON
data = json.loads(script_content)
data

{'name': 'Mercedes Benz Cla 180 1.3 Amg Line 2022',
 'image': 'https://http2.mlstatic.com/D_NQ_NP_718361-MCO77440081820_072024-O.webp',
 'offers': {'price': 129500000,
  'availability': 'http://schema.org/InStock',
  'url': 'https://carro.mercadolibre.com.co/MCO-2526957066-mercedes-benz-cla-180-13-amg-line-2022-_JM',
  '@type': 'Offer',
  'priceCurrency': 'COP',
  'priceValidUntil': '2024-07-17'},
 'brand': 'Mercedes-Benz',
 'sku': 'MCO2526957066',
 'color': 'Negro',
 '@context': 'http://schema.org',
 '@type': 'Vehicle',
 'itemCondition': 'https://schema.org/UsedCondition',
 'productID': 'MCO2526957066',
 'bodyType': 'Sedán',
 'fuelType': 'Gasolina',
 'numberOfDoors': '4',
 'vehicleEngine': '1300',
 'vehicleTransmission': 'Automática'}

➡️ Ahora vamos a crear un dataframe a partir de la información que conseguimos.

🗒️ **Observación:** Vamos a crear un dataframe donde una columna contenga el nombre de la característica y otra con los valores asociados.

In [8]:
features_ONLY_offers = list(data['offers'].keys())
features_ONLY_offers

['price', 'availability', 'url', '@type', 'priceCurrency', 'priceValidUntil']

In [9]:
features_NO_offers = list(data.keys())
features_NO_offers = [item for item in features_NO_offers if item != 'offers']
features_NO_offers

['name',
 'image',
 'brand',
 'sku',
 'color',
 '@context',
 '@type',
 'itemCondition',
 'productID',
 'bodyType',
 'fuelType',
 'numberOfDoors',
 'vehicleEngine',
 'vehicleTransmission']

In [10]:
features = features_ONLY_offers + features_NO_offers
features

['price',
 'availability',
 'url',
 '@type',
 'priceCurrency',
 'priceValidUntil',
 'name',
 'image',
 'brand',
 'sku',
 'color',
 '@context',
 '@type',
 'itemCondition',
 'productID',
 'bodyType',
 'fuelType',
 'numberOfDoors',
 'vehicleEngine',
 'vehicleTransmission']

➡️ Ahora deberíamos hacer lo mismo con los valores.

In [11]:
values_ONLY_offers = list(data['offers'].values())
values_NO_offers = [value for key, value in data.items() if key != 'offers']
values = values_ONLY_offers + values_NO_offers
values

[129500000,
 'http://schema.org/InStock',
 'https://carro.mercadolibre.com.co/MCO-2526957066-mercedes-benz-cla-180-13-amg-line-2022-_JM',
 'Offer',
 'COP',
 '2024-07-17',
 'Mercedes Benz Cla 180 1.3 Amg Line 2022',
 'https://http2.mlstatic.com/D_NQ_NP_718361-MCO77440081820_072024-O.webp',
 'Mercedes-Benz',
 'MCO2526957066',
 'Negro',
 'http://schema.org',
 'Vehicle',
 'https://schema.org/UsedCondition',
 'MCO2526957066',
 'Sedán',
 'Gasolina',
 '4',
 '1300',
 'Automática']

➡️ Convertimos en un dicionario las listas previamente definidas. Utilizamos la funcion 📦 *zip* para unir en pares las llaves y los valores. Adicionalmente, creamos un dataframe a partir de este diccionario utilizando el atributo **index=[0]** que nos ayuda a dejar como nombre de columnas las llaves con el valor respectivo.

In [12]:
data_dict = {str(feature): value for feature, value in zip(features, values)}
df = pd.DataFrame(data_dict, index=[0])

🗒️ **Observación:** No olvidemos agregar el registro ('timestamp') de la operación y la página de origen

In [13]:
df['Registro'] = pd.Timestamp.now()
df['Origen_individual'] = conexion_url
df

,price,availability,url,@type,priceCurrency,priceValidUntil,name,image,brand,sku,...,@context,itemCondition,productID,bodyType,fuelType,numberOfDoors,vehicleEngine,vehicleTransmission,Registro,Origen_individual
0,129500000,http://schema.org/InStock,https://carro.mercadolibre.com.co/MCO-25269570...,Vehicle,COP,2024-07-17,Mercedes Benz Cla 180 1.3 Amg Line 2022,https://http2.mlstatic.com/D_NQ_NP_718361-MCO7...,Mercedes-Benz,MCO2526957066,...,http://schema.org,https://schema.org/UsedCondition,MCO2526957066,Sedán,Gasolina,4,1300,Automática,2024-07-14 16:31:37.391388,https://articulo.tucarro.com.co/MCO-2526957066...
